In [78]:
class Node:
    def __init__(self, ID, nxt = None, prev = None):
        self.ID = ID
        self.data = dict()
        self.prev = prev
        self.fingerTable = [nxt]

    # Update the finger table of this node when necessary
    def updateFingerTable(self, dht, k):
        del self.fingerTable[1:]
        for i in range(1, k):
            self.fingerTable.append(dht.findNode(dht._startNode, self.ID + 2 ** i))

        
class DHT:
    # The total number of IDs available in the DHT is 2 ** k
    def __init__(self, k):
        self._k = k
        self._size = 2 ** k    
        self._startNode = Node(0, k)
        self._startNode.fingerTable[0] = self._startNode
        self._startNode.prev = self._startNode
        self._startNode.updateFingerTable(self, k)

    # Hash function used to get the ID
    def getHashId(self, key):
        return key % self._size

    # Get distance between to IDs
    def distance(self, n1, n2):
        if n1 == n2:
            return 0
        if n1 < n2:
            return n2 - n1
        return self._size - n1 + n2

    # Get number of nodes in the system
    def getNumNodes(self):
        if self._startNode == None:
            return 0
        node = self._startNode
        n = 1
        while node.fingerTable[0] != self._startNode:
            n = n + 1
            node = node.fingerTable[0]
        return n
    
    # Find the node responsible for the key
    def findNode(self, start, key):
        hashId = self.getHashId(key)
        curr = start
        numJumps = 0
        while True:
            if curr.ID == hashId:
                #print("number of jumps: ", numJumps)
                print(hashId)
                return curr
            if self.distance(curr.ID, hashId) <= self.distance(curr.fingerTable[0].ID, hashId):
                #print("number of jumps: ", numJumps)
                print(hashId)
                return curr.fingerTable[0]
            tabSize = len(curr.fingerTable)
            i = 0;
            nextNode = curr.fingerTable[-1]
            while i < tabSize - 1:
                if self.distance(curr.fingerTable[i].ID, hashId) < self.distance(curr.fingerTable[i + 1].ID, hashId):
                    nextNode = curr.fingerTable[i]
                i = i + 1
            curr = nextNode
            numJumps += 1
            print("-----------------------------------")

    # Look up a key in the DHT
    def lookup(self, start, key):
        nodeForKey = self.findNode(start, key)
        if key in nodeForKey.data:
            print("The ", key, " key is in node: ", nodeForKey.ID)
            return nodeForKey.data[key]
        return None

    # Store a key-value pair in the DHT
    def store(self, start, key, value):
        nodeForKey = self.findNode(start, key)
        nodeForKey.data[key] = value

    # When new node joins the system
    def join(self, newNode):
        # Find the node before which the new node should be inserted
        origNode = self.findNode(self._startNode, newNode.ID)

        print(origNode.ID, "  ", newNode.ID)
        # If there is a node with the same id, decline the join request for now
        if origNode.ID == newNode.ID:
            print("There is already a node with the same id!")
            return
        
        # Copy the key-value pairs that will belong to the new node after
        # the node is inserted in the system
        for key in origNode.data:
            hashId = self.getHashId(key)
            if self.distance(hashId, newNode.ID) < self.distance(hashId, origNode.ID):
                newNode.data[key] = origNode.data[key]

        # Update the prev and next pointers
        prevNode = origNode.prev
        newNode.fingerTable[0] = origNode
        newNode.prev = prevNode
        origNode.prev = newNode
        prevNode.fingerTable[0] = newNode
    
        # Set up finger table of the new node
        newNode.updateFingerTable(self, self._k)

        # Delete keys that have been moved to new node
        for key in list(origNode.data.keys()):
            hashId = self.getHashId(key)
            if self.distance(hashId, newNode.ID) < self.distance(hashId, origNode.ID):
                del origNode.data[key]
                
    
    def leave(self, node):
        # Copy all its key-value pairs to its successor in the system
        for k, v in node.data.items():
            node.fingerTable[0].data[k] = v
        # If this node is the only node in the system.
        if node.fingerTable[0] == node:
            self._startNode = None
        else:
            node.prev.fingerTable[0] = node.fingerTable[0]
            node.fingerTable[0] = prev = node.prev
            # If this deleted node was an entry point to the system, we
            # need to choose another entry point. Simply choose its successor
            if self._startNode == node:
                self._startNode = node.fingerTable[0]
    
    def updateAllFingerTables(self):
        self._startNode.updateFingerTable(self, self._k)
        curr = self._startNode.fingerTable[0]
        while curr != self._startNode:
            curr.updateFingerTable(self, self._k)
            curr = curr.fingerTable[0]

In [93]:
from random import randint

d = DHT(8)

# Add nodes
Nodelist = [0]
for i in range(5):
    r = randint(0, 256)
    Nodelist.append(r)
    print("create a node",r)
    d.join(Node(r))
    print("Node(r).ID",Node(r).ID)
print("Node in the Chord :",Nodelist)
d.updateAllFingerTables()


2
4
8
16
32
64
128
create a node 207
207
0    207
-----------------------------------
209
-----------------------------------
211
-----------------------------------
215
-----------------------------------
223
-----------------------------------
239
15
79
Node(r).ID 207
create a node 49
49
207    49
-----------------------------------
51
-----------------------------------
53
-----------------------------------
57
-----------------------------------
65
-----------------------------------
81
-----------------------------------
113
-----------------------------------
177
Node(r).ID 49
create a node 176
-----------------------------------
176
207    176
-----------------------------------
-----------------------------------
178
-----------------------------------
-----------------------------------
180
-----------------------------------
-----------------------------------
184
-----------------------------------
-----------------------------------
192
-----------------------------------
-

In [94]:
for i in range(5, 256, 10):
    print(i)
     # def store(self, start, key, value):
    d.store(d._startNode, i, i)

5
-----------------------------------
5
15
-----------------------------------
15
25
-----------------------------------
25
35
-----------------------------------
35
45
-----------------------------------
45
55
-----------------------------------
55
65
-----------------------------------
65
75
-----------------------------------
75
85
-----------------------------------
85
95
-----------------------------------
95
105
-----------------------------------
105
115
-----------------------------------
115
125
-----------------------------------
125
135
-----------------------------------
135
145
-----------------------------------
145
155
-----------------------------------
155
165
-----------------------------------
165
175
-----------------------------------
175
185
-----------------------------------
-----------------------------------
185
195
-----------------------------------
-----------------------------------
195
205
-----------------------------------
------------------------------

# Part 3: Correctly look up keys.Print the sequences of nodes get involved in this procedure. (40 pts)

In [95]:
# generate Key, and check key in which node. xiao qi gui, change the third parameter of for-loop can change key. 
print("Node in the Chord :",Nodelist)
for i in range(5, 256, 20):
    print("key",i)
    print(d.lookup(d._startNode, i))
    print("-------------------------------------")

Node in the Chord : [0, 207, 49, 176, 143, 1]
key 5
-----------------------------------
5
The  5  key is in node:  49
5
-------------------------------------
key 25
-----------------------------------
25
The  25  key is in node:  49
25
-------------------------------------
key 45
-----------------------------------
45
The  45  key is in node:  49
45
-------------------------------------
key 65
-----------------------------------
65
The  65  key is in node:  143
65
-------------------------------------
key 85
-----------------------------------
85
The  85  key is in node:  143
85
-------------------------------------
key 105
-----------------------------------
105
The  105  key is in node:  143
105
-------------------------------------
key 125
-----------------------------------
125
The  125  key is in node:  143
125
-------------------------------------
key 145
-----------------------------------
145
The  145  key is in node:  176
145
-------------------------------------
key 165
-----

# Part 4:[optional] ImplementNode::leave()correctly.(20pts)

In [92]:
#####leave(self, node)
print("Node in the Chord :",Nodelist)
#d.updateAllFingerTables()
#d.leave(Node(0))
for i in range(5, 256, 20):
    print("key",i)
    print(d.lookup(d._startNode, i))
    print("-------------------------------------")

Node in the Chord : [0, 155, 105]
key 5
5
The  5  key is in node:  105
5
-------------------------------------
key 25
25
The  25  key is in node:  105
25
-------------------------------------
key 45
45
The  45  key is in node:  105
45
-------------------------------------
key 65
65
The  65  key is in node:  105
65
-------------------------------------
key 85
85
The  85  key is in node:  105
85
-------------------------------------
key 105
-----------------------------------
105
The  105  key is in node:  105
105
-------------------------------------
key 125
-----------------------------------
125
The  125  key is in node:  155
125
-------------------------------------
key 145
-----------------------------------
145
The  145  key is in node:  155
145
-------------------------------------
key 165
-----------------------------------
165
The  165  key is in node:  0
165
-------------------------------------
key 185
-----------------------------------
185
The  185  key is in node:  0
185
--

# Shaffer space

In [91]:
print("initiate 2 space")
print("space 1:")
d1 = DHT(8)
print("------------------------")
print("space 2:")
d2 = DHT(8)

initiate 2 space
space 1:
2
4
8
16
32
64
128
------------------------
space 2:
2
4
8
16
32
64
128


In [101]:
print("space 1, create 5 node:")
NodeSpace1 =[]
for i in range(9):
    r = randint(0, 256)
    NodeSpace1.append(r)
    print("create a node",r)
    d1.join(Node(r))
    print("Node(r).ID",Node(r).ID)
print("Node in space 1 :",NodeSpace1)

space 1, create 5 node:
create a node 229
-----------------------------------
-----------------------------------
-----------------------------------
-----------------------------------
-----------------------------------
229
251    229
-----------------------------------
-----------------------------------
-----------------------------------
-----------------------------------
-----------------------------------
-----------------------------------
231
-----------------------------------
-----------------------------------
-----------------------------------
-----------------------------------
-----------------------------------
-----------------------------------
233
-----------------------------------
-----------------------------------
-----------------------------------
-----------------------------------
-----------------------------------
-----------------------------------
237
-----------------------------------
-----------------------------------
-------------------------------

In [102]:
print("space 2, create 5 node:")
NodeSpace2 =[]
for i in range(9):
    r = randint(0, 256)
    NodeSpace2.append(r)
    print("create a node",r)
    d2.join(Node(r))
    print("Node(r).ID",Node(r).ID)
print("Node in space 2 :",NodeSpace2)

space 2, create 5 node:
create a node 227
-----------------------------------
-----------------------------------
-----------------------------------
227
247    227
-----------------------------------
-----------------------------------
-----------------------------------
-----------------------------------
229
-----------------------------------
-----------------------------------
-----------------------------------
-----------------------------------
231
-----------------------------------
-----------------------------------
-----------------------------------
-----------------------------------
235
-----------------------------------
-----------------------------------
-----------------------------------
-----------------------------------
243
3
-----------------------------------
35
-----------------------------------
-----------------------------------
99
Node(r).ID 227
create a node 228
-----------------------------------
-----------------------------------
----------------------

In [114]:
from random import sample
# randomly generate 6 neighbors in space 1 and space 2
neighborsindexspace1 = random.sample(range(1,9), 6)
neighborsindexspace2 = random.sample(range(1,9), 6)
print("neighbors index in space1 :",neighborsindexspace1)
print("neighbors index in space2 :",neighborsindexspace2)

neighbors index in space1 : [5, 3, 4, 8, 1, 7]
neighbors index in space2 : [4, 6, 5, 3, 2, 8]


In [121]:
ds1 = []
for i in neighborsindexspace1:
    ds1.append(abs(NodeSpace1[i] - NodeSpace1[0])) 
print(ds1)

[198, 10, 4, 192, 79, 199]


In [123]:
ds2 = []
for i in neighborsindexspace2:
    ds2.append(abs(NodeSpace2[i] - NodeSpace2[0])) 
print(ds2)

[189, 201, 195, 82, 181, 220]


In [138]:
# distance function

def mindistanceIndex(neighborsindexspace,NodeSpace):
    ds1 = []
    for i in neighborsindexspace1:
        ds1.append(abs(NodeSpace1[i] - NodeSpace1[0])) 
    print(ds1)
    val, idx = min((val, idx) for (idx, val) in enumerate(ds1))
    print("xiao qi gui, minimum distance of the neighbor is :", val)
    print ("xiao qi gui, the index of minimum distance of the neighbor is :" ,idx)
    print ("xiao qi gui, i really confused what you want to print, it's just Intermediate variables")
    return idx

In [139]:
ds1f = mindistanceIndex(neighborsindexspace1,NodeSpace1)

[198, 10, 4, 192, 79, 199]
xiao qi gui, minimum distance of the neighbor is : 4
xiao qi gui, the index of minimum distance of the neighbor is : 2
xiao qi gui, i really confused what you want to print, it's just Intermediate variables
